In [1]:
import time
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.optim import LBFGS
#from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
import matplotlib.gridspec as gridspec


np.random.seed(1234)

torch.manual_seed(1234)            # 为CPU设置随机种子
torch.cuda.manual_seed(1234)       # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(1234)   # 为所有GPU设置随机种子


torch.set_default_dtype(torch.float64)
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
dtype  = torch.float64

In [2]:
class ConventBlock(nn.Module):
    def __init__(self,in_N,out_N):
        super(ConventBlock, self).__init__()
        self.Ls  = None
        self.net =nn.Sequential(nn.Linear(in_N,out_N),nn.Tanh())

    def forward(self, x):
        out = self.net(x)
        return out 

    
    
    
    
class Network(torch.nn.Module):
    def __init__(self,in_N,m,H_Layer,out_N,**kwargs):
        super(Network,self).__init__()
        self.xmu  = kwargs["xmean"]
        self.xstd = kwargs["xstdev"]
        self.tmu  = kwargs["tmean"]
        self.tstd = kwargs["tstdev"]
            
        layers = []
        layers.append(ConventBlock(in_N,m))
        for i in range(0,H_Layer-1):
            layers.append(ConventBlock(m,m))
         # output layer
        layers.append(nn.Linear(m,out_N))
        # total layers
        self.net = nn.Sequential(*layers)
        
    def forward(self,x,t):
        # normalize the input
        x = (x - self.xmu)/self.xstd
        t = (t - self.tmu)/self.tstd
        data = torch.cat((x,t),dim=1);
    
        out  = self.net(data)
        return out 
    
    
    
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight.data)
        nn.init.zeros_(m.bias)

In [3]:
x_lb=-15.0
x_rb=15.0
t_0 =0
t_f=1



domain = np.array([[x_lb,t_0],[x_rb,t_f]])
x_N = torch.rand(10000000)*(x_rb-x_lb) + x_lb
t_N = torch.rand(10000000)*(t_f-t_0) + t_0
kwargs ={"xmean":x_N.mean(), "xstdev":x_N.std(), "tmean":t_N.mean(), "tstdev":t_N.std()}  
print(kwargs)
model  = Network(in_N=2,m=100,H_Layer=6,out_N=2,**kwargs)
model.to(device)
model.apply(init_weights)
print(model)
print(model.xmu)
print(model.xstd)

{'xmean': tensor(0.0043), 'xstdev': tensor(8.6593), 'tmean': tensor(0.5001), 'tstdev': tensor(0.2887)}
Network(
  (net): Sequential(
    (0): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=2, out_features=100, bias=True)
        (1): Tanh()
      )
    )
    (1): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Tanh()
      )
    )
    (2): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Tanh()
      )
    )
    (3): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Tanh()
      )
    )
    (4): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Tanh()
      )
    )
    (5): ConventBlock(
      (net): Sequential(
        (0): Linear(in_features=100, out_features=100, bias=True)
        (1): Ta

In [4]:
# def u_exact(x,t):
# #     a = torch.tensor(6**0.5)
#     alpha=torch.tensor(-1)
#     beta=torch.tensor(-3)
#     gamma=torch.tensor(-1.0)
#     uo=torch.tensor(-1/5)
#     u = uo - alpha / (2 * beta) + (24 * uo * gamma * (4 * uo.pow(2) * beta.pow(2) * t.pow(2) - 2 * uo * beta * x.pow(2) - 3 * gamma)) / ((-4 * uo.pow(2) * beta.pow(2) * t.pow(2) - 2 * uo * beta * x.pow(2) + 3 * gamma).pow(2))
#     return u.to(device)


# def v_exact(x,t):
# #     a = torch.tensor(6**0.5)
#     alpha=torch.tensor(-1)
#     beta=torch.tensor(-3)
#     gamma=torch.tensor(-1.0)
#     uo=torch.tensor(-1/5)
#     v =-(192 * t * uo.pow(3) * x * beta.pow(2) * gamma) / ((-4 * uo.pow(2) * beta.pow(2) * t.pow(2) - 2 * uo * beta * x.pow(2) + 3 * gamma).pow(2))
#     return v.to(device)






In [5]:
def fetch_interior_data(domain,N_data):
    """
    Sampling collocation point:
    Args : 
        - domain :(numpy array) for the size of the domain
        - N_data :(int) number of points 
    out : a tensor of collocation points
    """
    dim = domain.shape[0]
    soboleng = torch.quasirandom.SobolEngine(dimension=dim,scramble=True)
    data     = soboleng.draw(N_data,dtype=dtype)*(domain[1] - domain[0]) + domain[0]
    x       = data[:,0][:,None]
    t       = data[:,1][:,None]
    return x.to(device),t.to(device)


def fetch_initial_data(domain,N_data):
    x_min    = domain[0][0]
    x_max    = domain[1][0]
    soboleng = torch.quasirandom.SobolEngine(dimension=1,scramble=True)
    x        = soboleng.draw(N_data,dtype=dtype)*(x_max - x_min) + x_min
    t        = (domain[0][1]) *torch.ones_like(x)
    return x.to(device),t.to(device)





def fetch_boundary_data(domain,N_data):
    # Left and right BC
    N_data   = N_data//2 
    
    x_min    = domain[0][0]
    x_max    = domain[1][0]
    
    t_min    = domain[0][1]
    t_max    = domain[1][1]
    
    soboleng = torch.quasirandom.SobolEngine(dimension=1,scramble=True)
    t        = soboleng.draw(N_data,dtype=dtype)*(t_max - t_min) + t_min
#     x_lb     = torch.full_like(t,x_max)
#     x_rb     = torch.full_like(t,x_max)
    
    E_bc     = torch.cat(( torch.full_like(t,x_max),t), dim = 1)
    W_bc     = torch.cat(( torch.full_like(t,x_min),t), dim = 1)
    
    data     = torch.cat((E_bc, W_bc), dim = 0)

    
    x        = data[:,0][:,None]
    t        = data[:,1][:,None]
    return x.to(device),t.to(device)

In [6]:
def physics_loss(model,x,t):
    u,v = model(x,t)[:,0:1], model(x,t)[:,1:2]
    u_x,u_t = torch.autograd.grad(u,(x,t), torch.ones_like(u), create_graph=True)
    v_x,v_t = torch.autograd.grad(v,(x,t), torch.ones_like(v), create_graph=True)
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
    u_xxx = torch.autograd.grad(u_xx, x, torch.ones_like(u_xx), create_graph=True)[0]
    u_xxxx = torch.autograd.grad(u_xxx, x, torch.ones_like(u_xxx), create_graph=True)[0]
    alpha = torch.tensor(-1)
    beta  = torch.tensor(-3)
    gamma = torch.tensor(-1.0)
    eq_a  = v_t + alpha * u_x + beta * (2 * u * u_x) + gamma * u_xxx
    eq_b  = v_x - u_t
    loss = eq_a.pow(2) + eq_b.pow(2)
    return loss
    
    
def uboundary_loss(model,x,t):
    u,v = model(x,t)[:,0:1], model(x,t)[:,1:2]
    u_x,u_t = torch.autograd.grad(u,(x,t), torch.ones_like(u), create_graph=True)
    loss1   = (u[0:(u.shape[0])//2,:] - u[(u.shape[0])//2:,:]).pow(2)
    loss2   = (u_x[0:(u_x.shape[0])//2,:] - u_x[(u_x.shape[0])//2:,:]).pow(2)
    loss3   = torch.zeros_like(loss1)
    loss4   = torch.zeros_like(loss2)
    loss5   = torch.cat([loss1,loss3],0)
    loss6   = torch.cat([loss2,loss4],0)
    return loss5, loss6
    

def uinitial_loss(model,x,t):
    u,v = model(x,t)[:,0:1], model(x,t)[:,1:2]
#     q_x,q_t = torch.autograd.grad(q,(x,t), torch.ones_like(q), create_graph=True)
#     r_x,r_t = torch.autograd.grad(r,(x,t), torch.ones_like(r), create_graph=True)
    loss  =  (u - torch.exp(-x.pow(2))).pow(2)
    return loss

def vinitial_loss(model,x,t):
    u,v = model(x,t)[:,0:1], model(x,t)[:,1:2]
#     q_x,q_t = torch.autograd.grad(q,(x,t), torch.ones_like(q), create_graph=True)
#     r_x,r_t = torch.autograd.grad(r,(x,t), torch.ones_like(r), create_graph=True)
    loss  =  (v - torch.ones_like(v)).pow(2)
    return loss
    

    
def l1_regularization(model, l1_alpha):
    regularization_loss = 0
    for param in model.parameters():
        regularization_loss += torch.sum(abs(param))
    return l1_alpha * regularization_loss

def l2_regularization(model, l2_alpha):
    regularization_loss = 0
    for param in model.parameters():
        regularization_loss += torch.sum((param)**2)
    return l2_alpha * regularization_loss

In [7]:
epochs          = 9000
disp            = 100
print_to_consol = True

model.apply(init_weights)

# update the optimizer
optimizer = LBFGS(model.parameters(),
                  line_search_fn="strong_wolfe"       # can be "strong_wolfe"
                 )


# initialize penalty parameter
mu           = torch.tensor(1.0)

# maximum penalty value for safeguarding
mu_max      = torch.tensor(1e6)

# l2 norm of constraints |C|_2
eta          = torch.tensor(0.0)

# penalty tolerance
epsilon      = torch.tensor(1e-8)



N_data     = 2000

# collocation points
x_dm,t_dm  = fetch_interior_data(domain,N_data)
x_dm       = x_dm.requires_grad_(True)
t_dm       = t_dm.requires_grad_(True)



N_u        = 200
# boundary points 
x_bc,t_bc  = fetch_boundary_data(domain,N_u)
x_bc       = x_bc.requires_grad_(True)
t_bc       = t_bc.requires_grad_(True)



N_u0       = 100
# 1st initial condition points
x_ic,t_ic  = fetch_initial_data(domain,N_u0)
x_ic       = x_ic.requires_grad_(True)
t_ic       = t_ic.requires_grad_(True)



# lagrange multiplier for the boundary constraints
Lambda_ub1  = torch.zeros_like(x_bc)
Lambda_ub2  = torch.zeros_like(x_bc)

Lambda_u0 = torch.zeros_like(x_ic)
Lambda_v0 = torch.zeros_like(x_ic)



print("=========LBFGS Start========")
for epoch in range(epochs):
    def closure():
        if torch.is_grad_enabled():
            model.train()
            optimizer.zero_grad()

        pde_loss     = physics_loss(model,x_dm,t_dm)
        
        ub_loss1, ub_loss2  = uboundary_loss(model,x_bc,t_bc)

        
        u0_loss     =  uinitial_loss(model,x_ic,t_ic)
        v0_loss     =  vinitial_loss(model,x_ic,t_ic)
        
        penalty     = (ub_loss1.T@ ub_loss1 + ub_loss2.T@ ub_loss2 + u0_loss.T@ u0_loss + v0_loss.T@ v0_loss).sum()
        
        
        loss        =  pde_loss.sum() +  (Lambda_ub1  * ub_loss1).sum() + (Lambda_ub2 * ub_loss2).sum() + (Lambda_u0 * u0_loss).sum() +  (Lambda_v0 * v0_loss).sum() + 0.5 * mu   * penalty + l1_regularization(model, 1e-5)
        
        if loss.requires_grad:
            loss.backward()
            
        return loss
    
    
    def _closure():
        model.eval()
      
        pde_loss     = physics_loss(model,x_dm,t_dm)
        
        ub_loss1, ub_loss2  = uboundary_loss(model,x_bc,t_bc)

        
        u0_loss     =  uinitial_loss(model,x_ic,t_ic)
        v0_loss     =  vinitial_loss(model,x_ic,t_ic)
        
        
        penalty     = (ub_loss1.T@ ub_loss1 + ub_loss2.T@ ub_loss2 + u0_loss.T@ u0_loss + v0_loss.T@ v0_loss).sum()
        
        return pde_loss, ub_loss1, ub_loss2, u0_loss, v0_loss, penalty
    
    
    optimizer.step(closure)
    pde_loss, ub_loss1, ub_loss2, u0_loss, v0_loss, penalty = _closure() 
    with torch.no_grad():
        if (torch.sqrt(penalty) >= 0.25*eta) and (torch.sqrt(penalty) > epsilon):
            mu          = min(mu*2.0, mu_max)
            Lambda_ub1  += mu*(ub_loss1)
            Lambda_ub2  += mu*(ub_loss2)
            Lambda_u0   += mu*(u0_loss)
            Lambda_v0   += mu*(v0_loss)
            
        eta  = torch.sqrt(penalty)
        
    if (epoch + 1)%disp == 0 and print_to_consol:
        print(f'epoch : {epoch+1:3d}, mu: {mu:0.1f}, sum of pde loss: {pde_loss.sum():2.3e}, penalty loss: {penalty.item():2.3e}')
            
            

=========LBFGS Start========
epoch : 100, mu: 1000000.0, sum of pde loss: 1.465e+00, penalty loss: 1.159e-09
epoch : 200, mu: 1000000.0, sum of pde loss: 1.128e+00, penalty loss: 2.800e-10
epoch : 300, mu: 1000000.0, sum of pde loss: 2.828e-01, penalty loss: 1.962e-10
epoch : 400, mu: 1000000.0, sum of pde loss: 3.547e-02, penalty loss: 1.040e-11
epoch : 500, mu: 1000000.0, sum of pde loss: 1.149e-02, penalty loss: 3.582e-13
epoch : 600, mu: 1000000.0, sum of pde loss: 4.932e-03, penalty loss: 5.482e-14
epoch : 700, mu: 1000000.0, sum of pde loss: 2.983e-03, penalty loss: 2.461e-14
epoch : 800, mu: 1000000.0, sum of pde loss: 1.974e-03, penalty loss: 2.050e-14
epoch : 900, mu: 1000000.0, sum of pde loss: 1.498e-03, penalty loss: 1.214e-14
epoch : 1000, mu: 1000000.0, sum of pde loss: 1.171e-03, penalty loss: 6.561e-15
epoch : 1100, mu: 1000000.0, sum of pde loss: 9.412e-04, penalty loss: 2.922e-15
epoch : 1200, mu: 1000000.0, sum of pde loss: 7.878e-04, penalty loss: 1.793e-15
epoch : 

In [8]:
def evaluate(model,domain):
    model.eval()
    x = np.linspace(domain[0][0],domain[1][0],201)
    t = np.linspace(domain[0][1],domain[1][1],201)

    x_star,t_star = np.meshgrid(x,t)
    x_test = torch.tensor(x_star.flatten()[:,None], requires_grad=True).to(device)
    t_test = torch.tensor(t_star.flatten()[:,None], requires_grad=True).to(device)

    u_pred = model(x_test,t_test)[:,0:1]
    u_pred = u_pred.detach().cpu().numpy()
    v_pred = model(x_test,t_test)[:,1:2]
    v_pred = v_pred.detach().cpu().numpy()
#     u_star = u_exact(x_test,t_test)
#     u_star = u_star.detach().cpu().numpy()
#     v_star = v_exact(x_test,t_test) 
#     v_star = v_star.detach().cpu().numpy()
    
    
    
    scipy.io.savemat('X.mat', {'array': x_star.flatten()[:, None]})
    scipy.io.savemat('T.mat', {'array': t_star.flatten()[:, None]})
    scipy.io.savemat('u_pred.mat', {'array': u_pred.flatten()[:, None]})
#    scipy.io.savemat('u_star.mat', {'array': u_star.flatten()[:, None]})
    scipy.io.savemat('v_pred.mat', {'array': v_pred.flatten()[:, None]})
#    scipy.io.savemat('v_star.mat', {'array': v_star.flatten()[:, None]})

    
    

    
#     l2_u   = np.linalg.norm(u_star - u_pred, 2)/np.linalg.norm(u_star, 2)
#     linf_u = max(abs(u_star - u_pred)).item()
    
    
#     l2_v   = np.linalg.norm(v_star - v_pred, 2)/np.linalg.norm(v_star, 2)
#     linf_v = max(abs(v_star - v_pred)).item()
    
#     return l2_u,l2_v,linf_u,linf_v

In [9]:
# checkpointing the model 
torch.save(model.state_dict(),f"gaussofBQ.pt")
# evaluation
evaluate(model,domain)
# print the l2 norms
# print(f"relative l2_u error :{l2_u:2.3e}, relative l2_v error :{l2_v:2.3e}, linf_u error : {linf_u :2.3e}, linf_v error : {linf_v :2.3e}")